In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists,druglists
from demres.common.process_pt_features import *
from demres.common.process_entries import *
from demres.demins.constants import Study_Design as sd
from demres.demins.statistical_functions import *
from common.helper_functions import *
from pprint import pprint
from IPython.display import display

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
pd.set_option('display.max_rows', None)

## Process raw CSV files

In [6]:
create_pegmed()

In [7]:
create_pegprod()

In [250]:
# create_prescriptions()

In [9]:
# create_consultations()

In [10]:
# create_clinicals()

In [11]:
# create_tests()

In [12]:
# create_referrals()

In [13]:
# create_immunisations()

In [14]:
# create_medcoded_entries()

## Create processed CVS files to share with DEMINS teams
*convert the newly created HDF files to CVS (for use in other projects)*

In [15]:
# prescriptions = pd.read_hdf('data/pt_data/processed_data/hdf/prescriptions.hdf')
# prescriptions.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/prescriptions.csv',index=False)

In [16]:
# consultations = pd.read_hdf('data/pt_data/processed_data/hdf/consultations.hdf')
# consultations.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/consultations.csv',index=False)

In [17]:
# immunisations = pd.read_hdf('data/pt_data/processed_data/hdf/immunisations.hdf')
# immunisations.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/immunisations.csv',index=False)

In [18]:
# clinicals = pd.read_hdf('data/pt_data/processed_data/hdf/clinicals.hdf')
# clinicals.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/clinicals.csv',index=False)

In [19]:
# tests = pd.read_hdf('data/pt_data/processed_data/hdf/tests.hdf')
# tests.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/tests.csv',index=False)

In [20]:
# referrals = pd.read_hdf('data/pt_data/processed_data/hdf/referrals.hdf')
# referrals.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/referrals.csv',index=False)

In [21]:
# medcoded_entries = pd.read_hdf('data/pt_data/processed_data/hdf/medcoded_entries.hdf')
# medcoded_entries.to_csv('data/pt_data/processed_data/to_share_with_DEMINS/medcoded_entries.csv',index=False)

## Create basic pt_features dataframe
*pt_features will contain all the variables (e.g. age, female gender, insomnia) used in the logistic regression*

In [7]:
# specify subtype of dementia we're interested in - 'vascular','alzheimers' or 'all_dementia'
subtype = 'all_dementia' 

In [9]:
all_encounters = get_all_encounters()

In [10]:
all_entries = get_all_entries(all_encounters)

In [298]:
pt_features = create_pt_features()

In [299]:
pt_features = get_index_date_and_caseness_and_add_final_dementia_subtype(all_entries,pt_features)

In [ ]:
# pt_features = only_include_specific_dementia_subtype(pt_features,subtype=subtype) #does nothing if we're interested in all dementia

In [300]:
pt_features = add_data_start_and_end_dates(all_encounters,pt_features,subtype)

calculating latest_sysdate
calculating earliest_sysdate
resampling all_encounters - may take some time...
locating converted codes
choosing most appropriate measure of data_start
removing patients without any events
There are 14 patients without any events


In [301]:
pt_features.head(15)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_end,data_start
0,57001,32,001,0,2006-12-05,True,19477.0,2013-05-17,1997-03-07
1,60001,24,001,0,NaT,False,NaN,2013-05-20,1997-03-07
2,149001,19,001,1,2004-07-02,True,7572.0,2004-11-17,1997-03-07
3,364001,37,001,1,NaT,False,NaN,2013-05-17,1997-03-07
4,432001,20,001,1,NaT,False,NaN,2013-04-15,1997-03-07
5,519001,27,001,1,NaT,False,NaN,2013-05-07,1997-03-07
6,543001,40,001,0,2006-02-09,True,9509.0,2006-10-05,1995-01-31
7,558001,37,001,1,NaT,False,NaN,2013-04-17,1997-03-07
8,644001,35,001,1,2008-08-12,True,1350.0,2013-05-17,1997-03-07
9,663001,18,001,1,NaT,False,NaN,2013-05-14,1997-03-07


In [302]:
for window in sd.exposure_windows:
    print(window['name'],' being matched')    
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
    pt_features = match_cases_and_controls(pt_features,window)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)

12_to_7  being matched
All patients 94500
Number of suitable cases 12216
Number of controls 47020
75714  patients being removed as unmatchable
10_to_5  being matched
All patients 94500
Number of suitable cases 18849
Number of controls 47020
64082  patients being removed as unmatchable
8_to_3  being matched
All patients 94500
Number of suitable cases 26042
Number of controls 47020
50622  patients being removed as unmatchable


## Add derived variables to pt_features 
*e.g. insomnia count, history of stroke, consultation count*

In [9]:
medcoded_entries = pd.read_hdf('data/pt_data/processed_data/hdf/medcoded_entries.hdf')

In [12]:
prescriptions = pd.read_hdf('data/pt_data/processed_data/hdf/prescriptions.hdf')

In [304]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+sd.exposure_windows[1]['name']+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [305]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+sd.exposure_windows[1]['name']+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
create_pdd_for_each_drug(prescriptions,druglists.all_druglists,pt_features,sd.exposure_windows[1])

CARBAMAZEPINE 	pdd: 329.571934268
GABAPENTIN 	pdd: 711.545805055
LAMOTRIGINE 	pdd: 187.326325773
PREGABALIN 	pdd: 169.484271346
SODIUM VALPROATE 	pdd: 721.028033855
VALPROATE SEMISODIUM 	pdd: 496.21653085
OXCARBAZEPINE 	pdd: 573.913043478
BRIVARACETAM 	No prescriptions found
ETHOSUXIMIDE 	pdd: 500.0
LEVETIRACETAM 	pdd: 502.131478084
OXCARBAZEPINE 	pdd: 573.913043478
TOPIRAMATE 	pdd: 91.4574845578
VIGABATRIN 	pdd: 1000.0
LITHIUM CITRATE 	pdd: 891.292307692
LITHIUM CARBONATE 	pdd: 464.293337538
ALPRAZOLAM 	pdd: 0.239361702128
CHLORDIAZEPOXIDE HYDROCHLORIDE 	pdd: 10.8586489863
CLOBAZAM 	pdd: 11.7033750594
CLONAZEPAM 	pdd: 0.823051344315
DIAZEPAM 	pdd: 4.66742792872
FLUNITRAZEPAM 	pdd: 1.0
FLURAZEPAM 	No prescriptions found
FLURAZEPAM HYDROCHLORIDE 	pdd: 27.2297441592
LOPRAZOLAM 	No prescriptions found
LORAZEPAM 	pdd: 1.46318357973
LORMETAZEPAM 	pdd: 0.865905116329
MIDAZOLAM 	No prescriptions found
MIDAZOLAM HYDROCHLORIDE 	No prescriptions found
NITRAZEPAM 	pdd: 5.97603790155
OXAZEPAM 	pdd

In [12]:
pd.options.mode.chained_assignment = None  # default='warn'

# Add condition status (e.g. insomnia count, presence of diabetes, presence of stroke)
for window in sd.exposure_windows:
    print(window['name'],'...')
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
#     pt_features = get_multiple_condition_statuses(pt_features,medcoded_entries,window,codelists.all_codelists)
#     pt_features = create_PDD_columns_for_each_pt(pt_features,window,druglists.all_druglists,prescriptions)
    print(pt_features.columns)
#     pt_features = get_consultation_count(pt_features,all_encounters,window)
#     pt_features = create_quantiles_and_booleans(pt_features)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)

12_to_7 ...
Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_end', 'data_start', 'matchid',
       'reason_for_removal', 'age_at_index_date',
       'non_insomnia_GP_consultations', 'mood_stabilisers_and_AEDs_100_pdds',
       'lithium_100_pdds', 'benzodiazepines_100_pdds', 'z_drugs_100_pdds',
       'antipsychotics_100_pdds', 'antidepressants_100_pdds', 'insomnia',
       'stroke', 'heart_failure', 'mental_illness', 'sleep_apnoea',
       'chronic_pulmonary_disease', 'epilepsy', 'insomnia_any',
       'insomnia_count:0', 'insomnia_count:1_5', 'insomnia_count:above_5',
       'non_insomnia_GP_consultations:0', 'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:above_1000', 'age_at_index_date:65-69',
       'age_at_index_date:70-74', 'age_at_index_date:75-79',
       'age_at_index_date:80-84', 'age_at_index_date